In [2]:
import pandas as pd
import numpy as np
import datetime   
from dateutil.relativedelta import relativedelta 
from tqdm import tqdm
from scipy import stats
import naverMap
import weatherAPI
import joblib

In [3]:
# 모델
model=joblib.load('model/xgboost.pkl')

In [4]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
# train_data
train_data=pd.read_csv("risk_data/Train_data_set.csv", encoding='utf-8-sig')

In [8]:
train_data.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,126690,0,-1.8,0.0,7.0
1,1,1,0,126690,10,-1.8,0.0,7.0
2,1,1,0,126690,20,-1.8,0.0,7.0
3,1,1,0,126690,30,-1.8,0.0,7.0
4,1,1,0,126690,40,-1.8,0.0,7.0


In [9]:
#인덱스 코드
index=pd.read_csv("risk_data/sido_sigun_index.csv", encoding='utf-8-sig')

In [10]:
index.head()

,시도,시군구,encoded_시도,encoded_시군구,index
0,서울특별시,종로구,0,0,0
1,서울특별시,중구,0,1,1
2,서울특별시,용산구,0,2,2
3,서울특별시,성동구,0,3,3
4,서울특별시,광진구,0,4,4


In [61]:
code=pd.read_csv("시군구코드_2019_v2.csv", encoding='utf-8-sig')

In [62]:
code.head()

,시군구코드,시도명,시군구명,시군구2명
0,11110,서울특별시,종로구,NaN
1,11140,서울특별시,중구,NaN
2,11170,서울특별시,용산구,NaN
3,11200,서울특별시,성동구,NaN
4,11215,서울특별시,광진구,NaN


In [27]:
# index 번호 찾기 함수
index.query("시도=='충청남도' and 시군구=='공주시'")

,시도,시군구,encoded_시도,encoded_시군구,index
115,충청남도,공주시,11,115,126


In [47]:
# 철새수 기존 데이터 활용 (실제에서는 실제 데이터 있으면 됨)
birds=pd.read_csv("risk_data/independent_variable/birds_risk_table_v2.csv", encoding='utf-8-sig')

In [48]:
birds.head()

,month,시도,철새수,발생건수,예측 발생건수,발생건수/예측 발생건수,위험도
0,1,강원도,626130,2,9,0.22,0
1,1,경기도,1242626,55,16,3.44,3
2,1,경상남도,347682,11,6,1.83,1
3,1,경상북도,554884,2,8,0.25,0
4,1,광주광역시,49915,0,2,0.00,0


In [49]:
# 날씨 기존 데이터 활용 (실제에서는 실시간 데이터로 활용할 예정)
weather=pd.read_csv("risk_data/independent_variable/Weather_risk_table_final.csv", encoding='utf-8-sig')

In [50]:
weather.head()

,시도,month,day,avg_temp(C),daily_rain(mm),avg_r_humidity(%),temp_index,rain_index,humidity_index,risk
0,강원영동,1,1,1.2,0.0,33.2,0.0,0.0,0.0,0.0
1,강원영동,1,2,3.2,0.0,34.7,0.0,0.0,0.0,0.0
2,강원영동,1,3,3.4,0.0,35.5,0.0,0.0,0.0,0.0
3,강원영동,1,4,4.2,0.0,39.0,0.0,0.0,0.0,0.0
4,강원영동,1,5,3.2,1.8,43.8,0.0,0.0,0.0,0.0


In [64]:
# 2021.09.01 시점 충청남도 공주시에서 HAPI 발생할 경우 테스트 data set (단일발생)
year=2021
month=9
day=1
place='충청남도 공주시'

yeongdong=['강릉시', '동해시', '속초시', '삼척시', '태백시', '고성군', '양양군']
yeongse=['원주시','춘천시','철원군','화천군','양구군','인제군','홍천군','평창군','정선군','영월군','횡성군']
temp=place.split()
place_index=index.query("시도==@temp[0] and 시군구==@temp[1]")['index'].values[0]
test_data=pd.DataFrame(index=code.index,columns=['year','month','day','시군구index','철새수','발생거리','평균온도','일강수량','위험도','code'])
test_data['year']=year
test_data['month']=month
test_data['day']=day
place_start=naverMap.Geocode(place)
if len(str(month))==1:
    w_month='0'+str(month)
if len(str(day))==1:
    w_day='0'+str(day)
w_date=str(year)+w_month+w_day

for i in range(len(test_data)):
    sido=code.loc[i,'시도명']
    sigun=code.loc[i,'시군구명']
    test_data.loc[i,'code']=code.loc[i,'시군구코드']
    
    # 시군구 인덱스 적용
    try:
        test_data.loc[i,'시군구index']=index.query("시도==@sido and 시군구==@sigun")['index'].values[0]
    except:
        print(sido, sigun)
        
    # 철새수 입력
    try:
        test_data.loc[i,'철새수']=birds.query("month==@month and 시도==@sido")['철새수'].values[0]
    except:
        test_data.loc[i,'철새수']=0
        
    # 기후데이터 입력
    if sido=='강원도' and sigun in yeongdong:
        r_sido='강원영동'
    elif sido=='강원도' and sigun in yeongse:
        r_sido='강원영서'
    elif sido in ['서울특별시','경기도','인천광역시']:
        r_sido='서울경기'
    elif sido in ['세종특별자치시','대전광역시']:
        r_sido='충청남도'
    elif sido in ['울산광역시','부산광역시']:
        r_sido='경상남도'
    elif sido in ['대구광역시']:
        r_sido='경상북도'
    elif sido in ['광주광역시']:
        r_sido='전라남도'
    else:
        r_sido=sido
    try:
        test_data.loc[i,'평균온도']=weather.query("시도==@r_sido and month==@month and day==@day")['avg_temp(C)'].values[0]
        test_data.loc[i,'일강수량']=weather.query("시도==@r_sido and month==@month and day==@day")['daily_rain(mm)'].values[0]
    except:
        print(r_sido)
        
    # 거리 데이터 입력
    if sigun=='세종시':
        sigun=''
    place_goal=naverMap.Geocode(sido+' '+sigun)
    test_data.loc[i,'발생거리']=naverMap.direction(place_start, place_goal)
    
# 위험도 계산 (모델 적용)
X=test_data[list(test_data.columns[1:-2])]
y_pred = model.predict(X.values)
test_data['위험도']=y_pred

경상남도 고성군


In [75]:
# 충청남도 공주시 위험도 10점 부여
display(code.query("시군구명=='공주시'"))
test_data.loc[151,'위험도']=10

,시군구코드,시도명,시군구명,시군구2명
151,44150,충청남도,공주시,NaN


In [81]:
test_data.to_csv("충청남도공주시_20210901발생.csv", index=False, encoding='utf-8-sig')